In [26]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [27]:
df = pd.read_csv('shark_clean.csv')

In [28]:
df.head()

,Unnamed: 0,Date,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Time,Species
0,0,09-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,Fatal,Y,10h45,Not Available
1,1,07-Oct-2020,2020.0,Unprovoked,USA,Florida,"Miami Beach, Miami-Dade County",Body Surfing,M,Survived,N,Afternoon,Blacktip shark
2,2,04-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,"Toms Surf break, Hamersley Pool, North Beach, ...",Surfing,M,No Injury,N,10h45,Bronze whaler 1.5m
3,4,20-Sep-2020,2020.0,Unprovoked,USA,Florida,"Sombero Key Light, Monroe County",Snorkeling,M,Survived,N,10h30,Bull shark
4,5,17-Sep-2020,2020.0,Provoked,AUSTRALIA,Queensland,Fraser Island,Fishing,M,Survived,N,16h00,"""whitetip shark"""


In [29]:
df.drop(['Unnamed: 0', 'Area', 'Fatal (Y/N)', 'Location', 'Time'], axis=1, inplace=True)

In [30]:
for i in df:
    print(i, '/n')
    
    print(df[i].value_counts(normalize=True).head())
    print('-----------------')

Date /n
1957    0.001528
1950    0.001189
1942    0.001189
1958    0.001189
1941    0.001189
Name: Date, dtype: float64
-----------------
Year /n
2015.0    0.022075
2017.0    0.021056
2016.0    0.020207
0.0       0.020207
2011.0    0.020037
Name: Year, dtype: float64
-----------------
Type /n
Unprovoked      0.797928
Provoked        0.099677
Watercraft      0.056546
Sea Disaster    0.038377
Invalid         0.004245
Name: Type, dtype: float64
-----------------
Country /n
USA                 0.365597
AUSTRALIA           0.213619
SOUTH AFRICA        0.088130
PAPUA NEW GUINEA    0.022075
NEW ZEALAND         0.021056
Name: Country, dtype: float64
-----------------
Activity /n
Surfing          0.169808
Swimming         0.134318
Fishing          0.074206
Not Available    0.065716
Spearfishing     0.055357
Name: Activity, dtype: float64
-----------------
Sex /n
M    0.886059
F    0.113941
Name: Sex, dtype: float64
-----------------
Injury /n
Survived     0.671761
Fatal        0.211411
No Injur

# Creating Categoricals

## Date Categoricals

In [31]:
df['Year'] = [i[-4:] for i in df['Date']]

In [32]:
for i, d in enumerate(df['Year']):
    try:
        df['Year'][i] = int(d)
    except:
        df['Year'][i] = 0

In [33]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [34]:
df['Month'] = 'NA'

In [35]:
for i, d in enumerate(df['Date']):
    for m in months:
        if d.find(m) != -1:
            s = d.find(m)
            df['Month'][i] = d[s:s+3]

In [36]:
m = pd.get_dummies(df['Month'])

In [37]:
df = pd.concat([df,m], axis=1)
df.drop(['Date', 'NA', 'Month'], axis=1, inplace=True)

## Type categoricals

In [38]:
def make_dummy(col):
    top_4_values = df[col].value_counts()[:4]
    other_values = df[col].value_counts()[4:]
    
    for name in top_4_values.keys():
        df.loc[df[col] == name, f'{col}: {name}'] =1
        
    for name in other_values.keys():
        df.loc[df[col] == name, f'{col}: other'] =1
    
    df.drop(col, axis=1, inplace=True)

In [39]:
make_dummy('Type')

## Other Categoricals 

In [40]:
categoricals = ['Country', 'Activity', 'Sex', 'Species ']
 for i in categoricals:
        make_dummy(i)


In [41]:
 df.fillna(0, inplace=True)

In [42]:
 df.to_csv('shark_model.csv')

# Model

In [43]:
X = df.drop(['Injury'], axis = 1)
y = df['Injury']

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [45]:
from sklearn.preprocessing import MinMaxScaler
mmscaler = MinMaxScaler()


X_train_minmax = mmscaler.fit_transform(X_train)
X_test_minmax = mmscaler.transform(X_test)

ValueError: could not convert string to float: 'AUSTRALIA'

In [25]:
df.head()

,Year,Injury,Month,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,Type: Unprovoked,Type: Provoked,Type: Watercraft,Type: Sea Disaster,Type: other,Country: USA,Country: AUSTRALIA,Country: SOUTH AFRICA,Country: PAPUA NEW GUINEA,Country: other,Activity: Surfing,Activity: Swimming,Activity: Fishing,Activity: Not Available,Activity: other,Sex: M,Sex: F,Species : Not Available,Species : White shark,Species : Tiger shark,Species : Bull shark,Species : other
0,2020,Fatal,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2020,Survived,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2020,No Injury,Oct,0,0,0,0,0,0,0,0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2020,Survived,Sep,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2020,Survived,Sep,0,0,0,0,0,0,0,0,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
